In [2]:
!pip install streamlit -q

In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip


--2020-09-12 09:37:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.225.89.236, 52.0.105.155, 3.229.211.71, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.225.89.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  73.6MB/s    in 0.2s    

2020-09-12 09:37:40 (73.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]



In [38]:
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://91c673ab7b22.ngrok.io


In [28]:
%%file hello.py

import streamlit as st
import pandas as pd

st.title("Hello world")
brain = pd.read_csv("https://raw.githubusercontent.com/rezpe/datos_viz/master/brain.csv")

#Mostrar tablas de datos
st.dataframe(brain.head())
st.table(brain.head())

#Mostrar texto
st.markdown("""
## Texto de ejemplo
```python
1+1
```
""")

# Gráficos
# Matplotlib
import matplotlib.pyplot as plt
plt.scatter(brain["Body Weight"],brain["Brain Weight"])
st.pyplot()

# Altair
import altair as alt
hist_brain = alt.Chart(brain).mark_bar().encode(
    x=alt.X('Brain Weight',bin=alt.Bin(maxbins=100)),
    y="count()"
).properties(
    width=300,
    height=150,
    title="Relación peso del cerebro y del cuerpo"
).interactive()

hist_body = alt.Chart(brain).mark_bar().encode(
    x=alt.X('Body Weight',bin=alt.Bin(maxbins=100)),
    y="count()"
).properties(
    width=300,
    height=150,
    title="Relación peso del cerebro y del cuerpo"
).interactive()

scatter_brain_body = alt.Chart(brain).mark_circle().encode(
    x='Body Weight',
    y='Brain Weight'
).properties(
    width=700,
    height=300,
    title="Relación peso del cerebro y del cuerpo"
).interactive()

comp_brain = (hist_brain|hist_body)&scatter_brain_body
st.write(comp_brain)

Overwriting hello.py


In [33]:
%%file hello.py

import streamlit as st 
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

st.title("Google Trends")

st.markdown("Hemos analizado las búsquedas de Google de 4 tecnologías:")

trends = pd.read_csv("https://raw.githubusercontent.com/rezpe/datos_viz/master/google_trends.csv")

select_term = alt.selection(type="single",encodings=["x"])

trend_line = alt.Chart(trends).mark_line().encode(
    x="yearmonth(date):T",
    y="mean(value)",
    color="search_term"
).transform_filter(
    select_term
)

trend_bar = alt.Chart(trends).mark_bar().encode(
    x="search_term",
    y="value",
    color="search_term",
    tooltip="search_term"
).properties(
    selection=select_term
)

st.write(trend_bar|trend_line)

st.markdown("Puede hacer zoom para ver el detalle:")

select_date = alt.selection(type="interval",encodings=["x"])

mini_trend_line = alt.Chart(trends).mark_line().encode(
    x="yearmonth(date):T",
    y="mean(value)",
    color="search_term"
).properties(
    height=100
).add_selection(
    select_date
)

trend_line = alt.Chart(trends).mark_line().encode(
    x="yearmonth(date):T",
    y="mean(value)",
    color="search_term"
).transform_filter(
    select_date
)

st.write(mini_trend_line&trend_line)


Overwriting hello.py


In [ ]:
!streamlit run hello.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.196.53.209:8501

/usr/lib/python3.6/threading.py:864: DtypeWarning: Columns (31,32,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


In [36]:
%%file hello.py

import streamlit as st 
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

st.title("Locales de Madrid")

st.markdown("Hemos obtenido la lista de locales de la comunidad de Madrid. Teclee el tipo de local que quiera buscar:")

texto = st.text_input("Insertar tipo","BAR")

locales = pd.read_csv("https://github.com/rezpe/datos_viz/blob/master/locales_madrid.csv?raw=true")

alt.data_transformers.disable_max_rows()

# filtrar locales

locales_filt = locales[locales["rotulo"].str.contains(texto)]

st.markdown(f"La comunidad de Madrid contiene: {len(locales_filt)} {texto}(s) ")

points = alt.Chart(locales_filt).mark_point().encode(
    latitude="lat",
    longitude="lon",
    color="desc_distrito_local"
)

# remote geojson data object
url_geojson = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/madrid.geojson'
data_geojson_remote = alt.Data(url=url_geojson, format=alt.DataFormat(property='features',type='json'))

# chart object
background = alt.Chart(data_geojson_remote).mark_geoshape(
).encode(
).properties(
)

st.write(background+points)


Overwriting hello.py


In [35]:
%%file hello.py

import streamlit as st 
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

st.title("GDS Life Expectancy")

st.markdown("We gathered stats on several countries to check relationships between GDP and life expectancy")

lifecountries = pd.read_csv("https://raw.githubusercontent.com/rezpe/datos_viz/master/lifecountries.csv")

st.write(alt.Chart(lifecountries).mark_circle().encode(
    x='Country GDP',
    y=alt.Y('Life Expectancy',scale=alt.Scale(zero=False)),
    color='Continent',
    tooltip='country',
    size='size'
))

st.markdown("We see that after a certain level, there's no significant increase in life expectancy")


Overwriting hello.py


In [ ]:
El problemna es que cada vez que se ejecuta tiene que bajarse todos los documentos. Esto se arregla con el decorador @st.bbchache

In [ ]:
%%file hello.py

import streamlit as st 
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

st.title("Locales de Madrid")

st.markdown("Hemos obtenido la lista de locales de la comunidad de Madrid. Teclee el tipo de local que quiera buscar:")

texto = st.text_input("Insertar tipo","BAR")

@st.cache
def get_locales():
  return pd.read_csv("https://github.com/rezpe/datos_viz/blob/master/locales_madrid.csv?raw=true")

locales = get_locales()

alt.data_transformers.disable_max_rows()

# filtrar locales

locales_filt = locales[locales["rotulo"].str.contains(texto)]

st.markdown(f"La comunidad de Madrid contiene: {len(locales_filt)} {texto}(s) ")

points = alt.Chart(locales_filt).mark_point().encode(
    latitude="lat",
    longitude="lon",
    color="desc_distrito_local"
)

# remote geojson data object
url_geojson = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/madrid.geojson'
data_geojson_remote = alt.Data(url=url_geojson, format=alt.DataFormat(property='features',type='json'))

# chart object
background = alt.Chart(data_geojson_remote).mark_geoshape(
).encode(
).properties(
)

st.write(background+points)


In [ ]:
opciones de streamlit

In [ ]:
%%file hello.py

import streamlit as st 
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import datetime

st.button('Say hello')

agree = st.checkbox('I agree')

genre = st.radio(
    "What's your favorite movie genre",
    ('Comedy', 'Drama', 'Documentary'))

option = st.selectbox(
     'How would you like to be contacted?',
     ('Email', 'Home phone', 'Mobile phone'))


options = st.multiselect(
     'What are your favorite colors',
     ['Green', 'Yellow', 'Red', 'Blue'],
     ['Yellow', 'Red'])

age = st.slider('How old are you?', 0, 130, 25)

color = st.select_slider(
     'Select a color of the rainbow',
     options=['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet'])

d = st.date_input(
     "When's your birthday",
     datetime.date(2019, 7, 6))


t = st.time_input('Set an alarm for', datetime.time(8, 45))

uploaded_file = st.file_uploader("Choose a CSV file", type="csv")


In [ ]:
%%file hello.py

import streamlit as st 
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import datetime
# Load the library with the iris dataset
from sklearn.datasets import load_iris
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

st.title("Iris Classifier")

# Create an object called iris with the iris data
iris = load_iris()

# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=-1, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(iris.data, iris.target)

st.markdown("Change the value of sliders to get predictions")

feature1 = st.slider('Feature 1', 0, 10)
feature2 = st.slider('Feature 2', 0, 10)
feature3 = st.slider('Feature 3', 0, 10)
feature4 = st.slider('Feature 4', 0, 10)

prediction = clf.predict([[feature1,feature2,feature3,feature4]])

st.write("The prediction is:"+str(prediction))


In [ ]:
%%file hello.py

import streamlit as st 
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import datetime
import pickle

st.title("Iris Classifier")

#Solution 2
@st.cache
def get_model():
  return pickle.load(open("irisrf.pickle","rb"))

clf = get_model()

st.markdown("Change the value of sliders to get predictions")

feature1 = st.slider('Feature 1', 0, 10)
feature2 = st.slider('Feature 2', 0, 10)
feature3 = st.slider('Feature 3', 0, 10)
feature4 = st.slider('Feature 4', 0, 10)

prediction = clf.predict([[feature1,feature2,feature3,feature4]])

st.write("The prediction is:"+str(prediction))
